In [1]:
import json, nltk, io, pickle
import numpy as np
from itertools import chain

### Read data

In [2]:
with open('/pio/data/data/squad/train-v1.1.json') as f:
    train = json.load(f)

In [3]:
with open('/pio/data/data/squad/dev-v1.1.json') as f:
    dev = json.load(f)

# Preprocess wikipedia negative examples

### Choose neg example for (almost) each question

In [ ]:
data_simple = []

for par in train['data']:    
    for con in par['paragraphs']:        
        for q in con['qas']:
            question = q['question']            
            answers = []
            
            for ans in q['answers']:
                text = ans['text']
                ans_start = ans['answer_start']
                
                answers.append((ans_start, text))
                
            data_simple.append([answers[0][1], question])

In [169]:
data_neg = []

for i in xrange(len(data_simple)):
    a, q = data_simple[i]
    found = False
    for _, p, _ in wiki_pars[i]:
        p = p.decode('utf8')
        if a not in p:
            found = True
            data_neg.append([i, q, p])
            break
    if not found and wiki_pars[i]:
        print "No negative example for question", i

No negative example for question 191
No negative example for question 200
No negative example for question 458
No negative example for question 764
No negative example for question 1090
No negative example for question 1653
No negative example for question 2491
No negative example for question 2626
No negative example for question 2628
No negative example for question 2976
No negative example for question 2988
No negative example for question 3040
No negative example for question 3286
No negative example for question 4510
No negative example for question 4729
No negative example for question 5195
No negative example for question 5231
No negative example for question 5411
No negative example for question 5620
No negative example for question 5686
No negative example for question 5699
No negative example for question 6329
No negative example for question 6847
No negative example for question 7242
No negative example for question 7437
No negative example for question 7455
No negative exam

In [103]:
len(data_neg), len(data_simple)

(84882, 87599)

In [143]:
with open('/pio/data/data/squad/wiki_negative_train/negative_paragraphs.pkl', 'w') as f:
    pickle.dump(data_neg, f)

### Create pkls for negative examples

In [146]:
from nltk import word_tokenize

In [171]:
for i in xrange(len(data_neg)):
    data_neg[i][1] = map(lower, word_tokenize(data_neg[i][1]))
    data_neg[i][2] = map(lower, word_tokenize(data_neg[i][2]))

In [172]:
for i in xrange(len(data_neg)):
    data_neg[i][2].append(not_a_word)

In [173]:
for i in xrange(len(data_neg)):
    data_neg[i][0] = [[len(data_neg[i][2]) - 1]]

In [174]:
train_wiki_neg_bin_feats = []

for _, q, c in data_neg:
    train_wiki_neg_bin_feats.append(make_bin_feats([q, c]))

In [179]:
train_wiki_neg_ascii_chars = []

chars = [unichr(i) for i in xrange(128)]
i_to_c = chars
c_to_i = {v:k for (k,v) in list(enumerate(chars))}

for _, q, x in data_neg:
    q_char = [[1] + [c_to_i.get(c, 0) for c in w] + [2] for w in q]
    x_char = [[1] + [c_to_i.get(c, 0) for c in w] + [2] for w in x[:-1]] + [[1, 3, 2]]
    train_wiki_neg_ascii_chars.append([q_char, x_char])

In [192]:
with open('/pio/data/data/squad/wiki_negative_train/train_neg_bin_feats.pkl', 'w') as f:
    pickle.dump(train_wiki_neg_bin_feats, f)
    
with open('/pio/data/data/squad/wiki_negative_train/train_neg_char_ascii.pkl', 'w') as f:
    pickle.dump(train_wiki_neg_ascii_chars, f)

In [204]:
train_neg_with_glove_vocab = []

for a, q, x in data_neg:
    q_num = [glove_w_to_i.get(w, 0) for w in q]
    x_num = [glove_w_to_i.get(w, 0) for w in x]
    train_neg_with_glove_vocab.append([a, q_num, x_num])

In [206]:
with open('/pio/data/data/squad/wiki_negative_train/train_neg_with_glove_vocab.pkl', 'w') as f:
    pickle.dump(train_neg_with_glove_vocab, f)

In [2]:
train_wiki_neg_bin_feats = np.load('/pio/data/data/squad/wiki_negative_train/train_neg_bin_feats.pkl')
train_wiki_neg_ascii_chars = np.load('/pio/data/data/squad/wiki_negative_train/train_neg_char_ascii.pkl')
train_neg_with_glove_vocab = np.load('/pio/data/data/squad/wiki_negative_train/train_neg_with_glove_vocab.pkl')

In [6]:
too_long = []

for i, (_, x) in enumerate(train_wiki_neg_ascii_chars):
    for w in x:
        if len(w) > 35:
            too_long.append(i)
            break

In [8]:
train_wiki_neg_bin_feats_cut = []
train_wiki_neg_ascii_chars_cut = []
train_neg_with_glove_vocab_cut = []

for i in xrange(len(train_neg_with_glove_vocab)):
    if i not in too_long:
        train_wiki_neg_bin_feats_cut.append(train_wiki_neg_bin_feats[i])
        train_wiki_neg_ascii_chars_cut.append(train_wiki_neg_ascii_chars[i])
        train_neg_with_glove_vocab_cut.append(train_neg_with_glove_vocab[i])

In [10]:
len(train_neg_with_glove_vocab_cut)

84833

In [11]:
with open('/pio/data/data/squad/wiki_negative_train/train_neg_bin_feats.pkl', 'w') as f:
    pickle.dump(train_wiki_neg_bin_feats_cut, f)
    
with open('/pio/data/data/squad/wiki_negative_train/train_neg_char_ascii.pkl', 'w') as f:
    pickle.dump(train_wiki_neg_ascii_chars_cut, f)
    
with open('/pio/data/data/squad/wiki_negative_train/train_neg_with_glove_vocab.pkl', 'w') as f:
    pickle.dump(train_neg_with_glove_vocab_cut, f)